In [ ]:
import pandas as pd
import scipy as sp
import numpy as np
import sklearn as sk
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import random
import torch
import math

from abc import ABC, abstractmethod

In [ ]:
train_df = pd.read_csv('mnist_train.csv', sep=',')

In [ ]:
test_df = pd.read_csv('mnist_test.csv', sep=',')

In [ ]:
train_df

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,1x10,1x11,1x12,1x13,1x14,1x15,1x16,1x17,1x18,1x19,1x20,1x21,1x22,1x23,1x24,1x25,1x26,1x27,1x28,2x1,2x2,2x3,2x4,2x5,2x6,2x7,2x8,2x9,2x10,2x11,...,27x17,27x18,27x19,27x20,27x21,27x22,27x23,27x24,27x25,27x26,27x27,27x28,28x1,28x2,28x3,28x4,28x5,28x6,28x7,28x8,28x9,28x10,28x11,28x12,28x13,28x14,28x15,28x16,28x17,28x18,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,147,252,42,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
59996,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
59997,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
59998,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
test_df

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,1x10,1x11,1x12,1x13,1x14,1x15,1x16,1x17,1x18,1x19,1x20,1x21,1x22,1x23,1x24,1x25,1x26,1x27,1x28,2x1,2x2,2x3,2x4,2x5,2x6,2x7,2x8,2x9,2x10,2x11,...,27x17,27x18,27x19,27x20,27x21,27x22,27x23,27x24,27x25,27x26,27x27,27x28,28x1,28x2,28x3,28x4,28x5,28x6,28x7,28x8,28x9,28x10,28x11,28x12,28x13,28x14,28x15,28x16,28x17,28x18,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9996,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9997,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9998,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
X = train_df.drop('label', axis=1)
y = train_df['label']

In [ ]:
X_test = test_df.drop('label', axis=1)
y_test = test_df['label']

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [ ]:
unique, counts = np.unique(y, return_counts=True)
dict(zip(unique, counts))

{0: 5923,
 1: 6742,
 2: 5958,
 3: 6131,
 4: 5842,
 5: 5421,
 6: 5918,
 7: 6265,
 8: 5851,
 9: 5949}

In [ ]:
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# X = scaler.fit_transform(X)

In [ ]:
X_test = scaler.transform(X_test)

In [ ]:
class EqualityFunction(torch.autograd.Function):

  def __call__(self, x):
    return x


In [ ]:
class LayerBase(ABC):
  def __init__(self, activation):
    self._activation = activation

  @abstractmethod
  def forward(self, X):
    pass
  
class HiddenEulerLayer(LayerBase):
  def __init__(self, n_neurons, activation, h=1e-3):
    super().__init__(activation)
    self.W = torch.empty(n_neurons, n_neurons)
    torch.nn.init.kaiming_uniform_(self.W, mode='fan_in', nonlinearity='relu')
    # self.W = torch.Tensor(
    #     np.random.normal(scale=1., size=(n_neurons, n_neurons))
    # )
    self._h = h

  def forward(self, X):
    return X + self._h * self._activation(X @ self.W)
  
class HiddenRungeLayer(LayerBase):
  def __init__(self, n_neurons, activation, h=1e-3):
    super().__init__(activation)
    self.W = torch.empty(n_neurons, n_neurons)
    torch.nn.init.kaiming_uniform_(self.W, mode='fan_in', nonlinearity='relu')
    # self.W = torch.Tensor(
    #     np.random.normal(scale=1., size=(n_neurons, n_neurons))
    # )
    self._h = h

  def forward(self, X):
    k1 = self._activation(X @ self.W)
    k2 = self._activation((X + self._h / 2 * k1) @ self.W)
    k3 = self._activation((X + self._h / 2 * k2) @ self.W)
    k4 = self._activation((X + self._h * k3) @ self.W)
    
    return X + self._h / 6 * (k1 + 2 * k2 + 2 * k3 + k4)

class ConvolutionalLayer(LayerBase):
  def __init__(self, n_neurons_in, n_neurons_out, activation, h=1e-3):
    super().__init__(activation)
    self.W = torch.empty(n_neurons_in, n_neurons_out)
    torch.nn.init.kaiming_uniform_(self.W, mode='fan_in', nonlinearity='relu')
    # self.W = torch.Tensor(
    #     np.random.normal(scale=1., size=(n_neurons_in, n_neurons_out))
    # )
    self._h = h

  def forward(self, X):
    return self._h * self._activation(X @ self.W)

In [ ]:
class DeepCNNBase(ABC):

  ACTIVASIONS = {
    'equality': EqualityFunction(),
    'relu': torch.nn.ReLU(),
    'sigmoid': torch.sigmoid,
    'th': torch.tanh,
    'softmax': torch.nn.LogSoftmax(dim=1),
    'leakyReLU': torch.nn.LeakyReLU(0.1)
  }

  def __init__(self, n_layers=50, activation='equality'):
    self._n_layers = n_layers
    if activation not in self.ACTIVASIONS:
      raise RuntimeError('''Invalid activation function name. 
      Use one of: {}'''.format(','.join(self.ACTIVASIONS.keys())))
    self._activation = self.ACTIVASIONS[activation]
    self._layers = []

  @abstractmethod
  def fit(self):
    pass
  
  @abstractmethod
  def predict(self):
    pass


In [ ]:
class DeepCNNClassifier(DeepCNNBase):

  METRIC_FUNCTIONS = {
    'accuracy': metrics.accuracy_score,
    'f1': metrics.f1_score,
    'precision': metrics.precision_score,
    'recall': metrics.recall_score,
  }

  OPTIMIZERS = {
      'rmsprop': torch.optim.RMSprop,
      'adam': torch.optim.Adam,
      'adamax': torch.optim.Adamax,
      'adamw': torch.optim.AdamW,
      'adadelta': torch.optim.Adadelta,
  }

  METHODS = {
      'euler': HiddenEulerLayer,
      'runge': HiddenRungeLayer
  }

  def __init__(self, n_layers=3, n_features=None, activation='equality', metric='accuracy', 
               max_iter=10, tol=1e-4, batch_size=-1, max_opt_iter=100,
               opt_tol=1e-5, optimizer='adam', lr=0.1, l2=1e-3, h=1e-3,
               method='euler', verbose=False):
    super().__init__(n_layers, activation)
    if metric not in self.METRIC_FUNCTIONS:
      raise RuntimeError('''Invalid metric function name. 
      Use one of: {}'''.format(','.join(self.METRIC_FUNCTIONS.keys())))
    self._metric = self.METRIC_FUNCTIONS[metric]
    if optimizer not in self.OPTIMIZERS:
      raise RuntimeError('''Invalid optimizer name. 
      Use one of: {}'''.format(','.join(self.OPTIMIZERS.keys())))
    if method not in self.METHODS:
      raise RuntimeError('''Invalid discretization method name. 
      Use one of: {}'''.format(','.join(self.METHODS.keys())))
    self._optimizer = self.OPTIMIZERS[optimizer]
    self._layerclass = self.METHODS[method]
    self._max_iter = max_iter
    self._tol = tol
    self._verbose = verbose
    self._batch_size = batch_size
    self._max_opt_iter = max_opt_iter
    self._opt_tol = opt_tol
    self._lr = lr
    self._l2 = l2
    self._n_features = n_features
    self._h = h

  def fit(self, X, y, X_test=None, Y_test=None):
    self.metrics = []
    X = torch.Tensor(X)

    classes_count = np.unique(y).shape[0]
    y = torch.Tensor(y.reshape((y.shape[0]))).type(torch.LongTensor)

    self._layers = []
    input_features_count = X.shape[1] 
    features_count = self._n_features or X.shape[1]
    # Создаем слои нейронной сети
    self._layers.append(ConvolutionalLayer(input_features_count, features_count, self.ACTIVASIONS['equality'], h=self._h))
    for _ in range(self._n_layers):
      self._layers.append(self._layerclass(features_count, self._activation, h=self._h))
    # Выходной слой
    self._layers.append(ConvolutionalLayer(features_count, classes_count, self.ACTIVASIONS['softmax'], h=self._h))
    # Обучение
    iter_count = 0
    prev_metric = None
    while iter_count < self._max_iter:
      lr = self.__get_lr(iter_count)
      for stop_layer_idx in range(len(self._layers) - 1, -1, -1):
        # Вот эту логику полезно использовать, когда размер батчей большой
        # X_copy = X.clone().detach() 
        # Forward propagation до еще не оптимизированного слоя (их берем с конца)
        # X_copy = self.__forward(X_copy, to_i=stop_layer_idx)
        # Оптимизируем
        self.__optimize(stop_layer_idx, X, y, lr)
      # Тут полный forward на оптимизированной сети FIXME
      X_copy = X.clone().detach()
      X_copy = self.__forward(X_copy)
      y_pred = self.__out_to_labels(X_copy)
      metric = self._metric(y, y_pred)
      if X_test is not None and Y_test is not None:
        y_test_pred = self.predict(X_test)
        metric_test = self._metric(y_test_pred, Y_test)
        print('precision',metrics.precision_score(y_test_pred, Y_test, average='weighted'))
        print('recall', metrics.recall_score(y_test_pred, Y_test, average='weighted'))
        self.metrics.append((metric, metric_test))
      if self._verbose:
        print('=' * 10)
        print('Temp iteration: {}'.format(iter_count + 1))
        print('Metric: {}'.format(metric))
        if X_test is not None and Y_test is not None:
          print('Test metric: {}'.format(metric_test))
        print('=' * 10 + '\n')
      
      if prev_metric is not None and np.abs(metric - prev_metric) < self._tol:
        break 
      prev_metric = metric
      iter_count += 1
  
  def predict(self, X):
    X_copy = torch.Tensor(X)
    X_copy = self.__forward(X_copy)
    return self.__out_to_labels(X_copy)
  
  def score(self, X, y):
    y_pred = self.predict(X)
    return self._metric(y, y_pred)

  def __forward(self, X, from_i=None, to_i=None):
    if from_i is None:
      from_i = 0
    if to_i is None:
      to_i = len(self._layers)
    for i in range(from_i, to_i):
      X = self._layers[i].forward(X)
    return X

  def __get_lr(self, iter):
    drop = 0.7
    epochs_drop = 5.0
    return self._lr * math.pow(drop, math.floor((1+iter)/epochs_drop))

  def __optimize(self, layer_idx, X, y, lr):
    iter_count = 0
    prev_error = None
    self._layers[layer_idx].W.requires_grad = True

    optimizer = self._optimizer([self._layers[layer_idx].W], lr=lr,
                                weight_decay=self._l2)
    scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=1e-4, max_lr=0.1,
                                                  step_size_up=self._max_opt_iter/2)
    
    while iter_count < self._max_opt_iter:
      optimizer.zero_grad(set_to_none=True)
      # Градиент на текущем слое
      if self._batch_size >= X.shape[0] or self._batch_size == -1:
        indicies = np.arange(X.shape[0])
      else:
        indicies = random.sample_without_replacement(n_population=X.shape[0], 
                                                     n_samples=self._batch_size)
      X_copy = X[indicies].clone().detach()
      X_copy = self.__forward(X_copy)
      # Кросс-энтропия
      loss = torch.nn.NLLLoss()
      error = loss(X_copy, y[indicies])
      # if prev_error is not None:
      #   print(np.abs(prev_error - error.item()))
      if prev_error is not None and np.abs(prev_error - error.item()) < self._opt_tol:
        break
      error.backward()
      optimizer.step()
      scheduler.step()
      # print(self._layers[layer_idx].W.grad)
      prev_error = error.item()
      
      iter_count += 1
    print(self._layers[layer_idx].W.abs().max(), lr, optimizer.param_groups[0]['lr'])
    optimizer.zero_grad(set_to_none=True)
    self._layers[layer_idx].W.requires_grad = False
    if self._verbose:
      if layer_idx == 0:
        print('Error on input layer: ', end='')
      elif layer_idx == len(self._layers) - 1:
        print('Error on output layer: ', end='')
      else:
        print('Error on layer {}: '.format(layer_idx), end='')
      print('{} (iter_count: {})'.format(prev_error, iter_count))

  def __out_to_labels(self, X_out):
    X_out = X_out.detach().numpy()
    return np.argmax(X_out, axis=1)


In [ ]:
clf = DeepCNNClassifier(verbose=True,
                        n_layers=2,
                        n_features=800,
                        batch_size=700, 
                        max_opt_iter=100, 
                        max_iter=45,
                        activation='th',
                        optimizer='rmsprop',
                        method='euler',
                        tol=-1,
                        opt_tol=-1,
                        lr=1e-2, 
                        l2=1e-2,
                        h=4)

In [ ]:

%prun clf.fit(X, y.to_numpy(), X_test, y_test)


tensor(4.0846, grad_fn=<MaxBackward1>) 0.01 0.0001
Error on output layer: 57.095272064208984 (iter_count: 100)
tensor(4.6415, grad_fn=<MaxBackward1>) 0.01 0.0001
Error on layer 2: 51.4890251159668 (iter_count: 100)
tensor(4.3178, grad_fn=<MaxBackward1>) 0.01 0.0001
Error on layer 1: 39.51609802246094 (iter_count: 100)
tensor(6.5086, grad_fn=<MaxBackward1>) 0.01 0.0001
Error on input layer: 2318.638671875 (iter_count: 100)
precision 0.9107735649618341
recall 0.91
Temp iteration: 1
Metric: 0.90985
Test metric: 0.91

tensor(18.1018, grad_fn=<MaxBackward1>) 0.01 0.0001
Error on output layer: 5086.2373046875 (iter_count: 100)
tensor(6.4926, grad_fn=<MaxBackward1>) 0.01 0.0001
Error on layer 2: 3032.861328125 (iter_count: 100)
tensor(5.9515, grad_fn=<MaxBackward1>) 0.01 0.0001
Error on layer 1: 3189.615478515625 (iter_count: 100)
tensor(10.0356, grad_fn=<MaxBackward1>) 0.01 0.0001
Error on input layer: 7988.9599609375 (iter_count: 100)
precision 0.9090167291380045
recall 0.9086
Temp iteratio

KeyboardInterrupt: ignored

ReLu: Temp iteration: 45
Metric: 0.98705
Test metric: 0.9632

In [ ]:
clf._layers[0].W

tensor([[-0.0676, -0.0580,  0.0018,  ...,  0.0199, -0.0046,  0.0680],
        [ 0.0788, -0.0698, -0.0751,  ...,  0.0392,  0.0803, -0.0178],
        [-0.0784,  0.0162, -0.0012,  ...,  0.0268, -0.0042, -0.0630],
        ...,
        [-0.0760,  0.0694, -0.0179,  ...,  0.0040, -0.0100,  0.0529],
        [ 0.0404, -0.0068, -0.0602,  ...,  0.0170, -0.0480,  0.0577],
        [-0.0411, -0.0788,  0.0522,  ...,  0.0576,  0.0307, -0.0012]])

In [ ]:
clf.score(X_test, y_test)

0.9771

In [ ]:
from PIL import Image
col = Image.open("train2.png")
col = col.convert('L')
col

In [ ]:
img = np.asarray(col).copy()

In [ ]:
img = scaler.transform([img.flatten()])

In [ ]:
clf.predict(img)

array([5])

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, max_iter=300).fit(X, y)


In [ ]:
y_pred = clf.predict(X_test)


In [ ]:
metrics.accuracy_score(y_test, y_pred)

0.9214

In [ ]:
x = torch.Tensor(X[0:3])
y_true = np.array(y[0:3])
y_true = torch.Tensor(y_true.reshape((y_true.shape[0]))).type(torch.LongTensor)
W = torch.Tensor(
        np.random.normal(scale=1., size=(784, 300))
)
W2 = torch.Tensor(
        np.random.normal(scale=1., size=(300, 10))
)

W2.requires_grad=True
sm = torch.nn.LogSoftmax(dim=1)
out = sm(x @ W @ W2)
loss = torch.nn.NLLLoss()
error = loss(out, y_true)


optimizer = torch.optim.Adam([W], lr=0.1)

In [ ]:
optimizer.zero_grad()
error.backward()
print(W2.grad)
optimizer.step()
print(W2.grad)

tensor([[  2.4660,   0.0000,   6.8226,  ...,   0.0000,  -2.4660,   0.0000],
        [ -2.0428,   0.0000,   0.1258,  ...,   0.0000,   2.0428,   0.0000],
        [ 16.2204,   0.0000,   4.6968,  ...,   0.0000, -16.2204,   0.0000],
        ...,
        [  5.4201,   0.0000,   9.2073,  ...,   0.0000,  -5.4201,   0.0000],
        [  2.4052,   0.0000,   0.6211,  ...,   0.0000,  -2.4052,   0.0000],
        [ -3.7843,   0.0000,  -1.0500,  ...,   0.0000,   3.7843,   0.0000]])
tensor([[  2.4660,   0.0000,   6.8226,  ...,   0.0000,  -2.4660,   0.0000],
        [ -2.0428,   0.0000,   0.1258,  ...,   0.0000,   2.0428,   0.0000],
        [ 16.2204,   0.0000,   4.6968,  ...,   0.0000, -16.2204,   0.0000],
        ...,
        [  5.4201,   0.0000,   9.2073,  ...,   0.0000,  -5.4201,   0.0000],
        [  2.4052,   0.0000,   0.6211,  ...,   0.0000,  -2.4052,   0.0000],
        [ -3.7843,   0.0000,  -1.0500,  ...,   0.0000,   3.7843,   0.0000]])


In [ ]:
torch.norm(clf._layers[2].W, 2)

tensor(799.4763)

In [ ]:
x @ W @ W2

tensor([[-1408.2926,  -200.2903,  1497.4525,   266.7481,   -87.7223,   -92.8850,
           717.7164,   202.9604,   -66.0361,  -581.4584],
        [ -148.4885,  -195.0614,  -774.4548,  -175.8048,   -61.8154,    41.5983,
           447.3047,  -616.2546,   509.8233,  -780.6782],
        [  631.0779,  -199.2090,   802.9531, -1436.6494,   735.5991,   465.0322,
          1744.3058,   119.4763, -1334.6051,  -679.0886]],
       grad_fn=<MmBackward>)